In [36]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the results of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [5]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [4]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [7]:
data[:10]

[{'reviewerID': 'A238V1XTSK9NFE',
  'asin': 'B00004VX3T',
  'reviewerName': 'Andrew Lynn',
  'helpful': [2, 2],
  'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automa

In [67]:
data[0]['overall']

5.0

In [68]:
len(data)

60000

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [5]:
ratings = [ data[i]['overall'] for i in range(0,len(data))]
ratings[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [28]:
import spacy

nlp=spacy.load('en')

In [71]:
# def reviews_to_docs(data):
#     sentences=[]
#     for i in range(0,len(data)):
#         sentences+=nlp(data[i]['reviewText'])
#     return sentences
        
# reviews_to_docs(data)[:10]

In [6]:
from sklearn.base import BaseEstimator,TransformerMixin

class KeyTransformer(BaseEstimator,TransformerMixin):
    def __init__(self,key):
        self.key=key
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        return  [ d[self.key] for d in X]

In [7]:
kt=KeyTransformer('reviewText')
x=kt.fit_transform(data)
x[:5]

["I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatically supports wheel mice.  Just plug it in and use it - it's that easy.",
 'One by one, all of the discs went bad within a 6

In [74]:
CountVectorizer?

In [75]:
print(len(x))
print(len(ratings))

60000
60000


In [8]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from spacy.lang.en import STOP_WORDS
from sklearn.linear_model import SGDRegressor,Ridge
from sklearn.pipeline import Pipeline


bag_of_words=HashingVectorizer(stop_words=STOP_WORDS)

# pipe=Pipeline([('BOW',bag_of_words),('reg',SGDRegressor())])
# pipe.fit(x, ratings)

# print("Training accuracy: {}".format(pipe.score(x, ratings)))

In [80]:

bag_of_words_model = Pipeline([('kt',KeyTransformer('reviewText')),('BOW',bag_of_words),('reg',Ridge())])
bag_of_words_model.fit(data,ratings)

print("Training accuracy: {}".format(bag_of_words_model.score(data, ratings)))

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Training accuracy: 0.5501193017730213


In [81]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score: 1.761


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalization to your model to improve performance.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
TfidfVectorizer?

In [16]:
tfidf = TfidfVectorizer()
normalized_model = Pipeline([('kt',KeyTransformer('reviewText')),('tfidf',tfidf),('reg',Ridge())])
normalized_model.fit(data,ratings)

print("Training accuracy: {}".format(normalized_model.score(data, ratings)))

Training accuracy: 0.6082635357154005


In [17]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score: 1.059


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [10]:
hash_bi = HashingVectorizer(ngram_range=(1,2))
bigrams_model = Pipeline([('kt',KeyTransformer('reviewText')),('hash_bi',hash_bi),('reg',Ridge())])

In [11]:
bigrams_model.fit(data,ratings)

print("Training accuracy: {}".format(bigrams_model.score(data, ratings)))

Training accuracy: 0.6982230059362742


In [12]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score: 1.152


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the word "terrible" appears in the text, given that the review is negative. Using these probabilities, we can derive a **polarity score** for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [15]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

--2021-05-09 08:03:08--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.163.217
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.163.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970853 (2.8M) [application/x-gzip]
Saving to: ‘./data/amazon_one_and_five_star_reviews.json.gz’

     0K .......... .......... .......... .......... ..........  1% 3.45M 1s
    50K .......... .......... .......... .......... ..........  3% 6.83M 1s
   100K .......... .......... .......... .......... ..........  5% 57.4M 0s
   150K .......... .......... .......... .......... ..........  6% 7.35M 0s
   200K .......... .......... .......... .......... ..........  8% 60.1M 0s
   250K .......... .......... .......... .......... .......... 10% 56.1M 0s
   300K .......... .......... .......... .......... .....

In order to avoid memory issues, let's delete the older data.

In [16]:
del data, ratings

In [19]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = ...


In [24]:
data_polarity[:2]

[{'reviewerID': 'A31RSJTGLVV3TR',
  'asin': 'B000093UDP',
  'reviewerName': 'T. Wayne',
  'helpful': [11, 12],
  'reviewText': "This worked perfectly for about 8 rewinds.  But once it eats one of your precious tapes you'll probably be as dissapointed as I was.  Buyer beware... these rewinders are mostly the same look; but marketed by different companies.  They all eat tape.",
  'overall': 1.0,
  'summary': "It's a super tape-eater",
  'unixReviewTime': 1210550400,
  'reviewTime': '05 12, 2008'},
 {'reviewerID': 'A2Y739CRM15WDL',
  'asin': 'B00005MNSR',
  'reviewerName': 'Sires "I like mysteries (particularly British...',
  'helpful': [0, 0],
  'reviewText': "This was a choice in my gold box or I might not have bought it. I would have been missing out on a very good monitor if I hadn't.I just bought a new 2.53 gigahertz multi media computer running XP home edition and wanted a monitor with a small footprint but still able to handle graphics intense functions.  Set up was a breeze.  I ju

In [23]:
ratings=[X['overall'] for X in data_polarity]
ratings[:2]

[1.0, 5.0]

In [29]:
stop_words_str = " ".join(STOP_WORDS)
stop_words_lemma = set(word.lemma_.lower() for word in nlp(stop_words_str))

In [33]:
def lemmatizer(text):
    return [word.lemma_.lower() for word in nlp(text)]

In [34]:
tfidf = TfidfVectorizer(stop_words=stop_words_lemma, tokenizer=lemmatizer, ngram_range=(1, 1))
most_polar = Pipeline([('kt',KeyTransformer('reviewText')),('vectorizer', tfidf), ('classifier', MultinomialNB())])
most_polar.fit(data_polarity, ratings)

print("Training accuracy: {}".format(most_polar.score(data_polarity, ratings)))

Training accuracy: 0.8994


In [37]:
#get features (vocab) from model
feat_to_token = most_polar['vectorizer'].get_feature_names()

#get the log probability from model
log_prob = most_polar['classifier'].feature_log_prob_

#collapse log probability into one row
polarity = log_prob[0, :] - log_prob[1, :]

#combine polarity and feature names
most_polar = sorted(list(zip(polarity, feat_to_token)))

In [38]:
n=25
most_polar = most_polar[:n] + most_polar[-n:]

In [39]:
top_50 = [term for score, term in most_polar]

In [40]:
grader.score.nlp__most_polar(top_50)

Your score: 0.900


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*